<a href="https://colab.research.google.com/github/anedun2/Analytics-in-Big-Data/blob/master/text_count_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!ls

In [0]:
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!ls 

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
df = spark.createDataFrame([{"Google": "Colab","Spark": "Scala"} ,{"Google": "Dataproc","Spark":"Python"}])
df.show()

In [0]:
import string # to remove punctuation
import re
import pandas as pd
import os
import itertools
from collections import Counter
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, col


# Step 1: Load data

In [0]:
#importing file
from google.colab import files
uploaded = files.upload()

Saving Amazon_Responded_Oct05.csv to Amazon_Responded_Oct05.csv


In [0]:
!ls

Amazon_Responded_Oct05.csv  spark-2.4.4-bin-hadoop2.7.tgz
sample_data		    spark-2.4.4-bin-hadoop2.7.tgz.1
spark-2.4.4-bin-hadoop2.7


In [0]:
#!rm 'Amazon_Responded_Oct05 (1).csv'

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
dataset = spark.read.csv('Amazon_Responded_Oct05.csv',inferSchema=True, header =True)

In [0]:
dataset.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- tweet_created_at: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_statuses_count: string (nullable = true)
 |-- user_favourites_count: string (nullable = true)
 |-- user_protected: string (nullable = true)
 |-- user_listed_count: string (nullable = true)
 |-- user_following: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_friends_count: string (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- tweet_language: string (nullable = true)
 |-- text_: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- favorited: string (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id_str: string (nullable = true)
 |--

# Step 2: Extract the three columns

In [0]:
d1 = dataset.select("user_id_str", "user_followers_count", "text_")
d1 = d1.withColumn("user_followers_count", d1["user_followers_count"].cast(IntegerType()))

d1

DataFrame[user_id_str: string, user_followers_count: int, text_: string]

In [0]:
print((d1.count(), len(d1.columns)))

(413247, 3)


In [0]:
d2 = d1.na.fill({'user_id_str': '', 'user_followers_count': 0, 'text_': ''})

# Step 3: Remove the duplicated records

In [0]:
#d1 = d1.dropna()
#df.na.fill(0).show()
#d3 = d2.orderBy(d2.user_followers_count.desc())
d3 = d2.orderBy('user_followers_count', ascending=False).dropDuplicates(subset=['user_id_str'])

print((d3.count(), len(d3.columns)))

#d1.orderBy(["user_id_str", "user_followers_count"], ascending=[0, 1]).collect()
#d1.orderBy(["user_followers_count"], ascending=[0]).collect()

(74134, 3)


In [0]:
d3.show()

+-----------+--------------------+--------------------+
|user_id_str|user_followers_count|               text_|
+-----------+--------------------+--------------------+
| 1004756264|                 125|I ordered somethi...|
| 1022288054|                 284|@amazon are horre...|
| 1045337910|                  12|poor service agai...|
|   10475672|                   0|                    |
|   10723382|               19648|@AmazonHelp A pac...|
| 1081442186|                  97|Never use Amazon ...|
|       1090|                   0|                    |
| 1096495627|                5758|@AmazonHelp the a...|
|  111115704|                 114|@JeffBezos thanks...|
|  111353755|                  77|@amazonIN failed ...|
|   11284752|                3366|@AmazonHelp what ...|
|  115422663|                 399|When you order a ...|
|  115546961|                 512|@amazonIN Does an...|
|   11571722|                 231|~@AmazonHelp I bo...|
|  117769974|                 415|@AmazonHelp ho

In [0]:
#d3.orderBy('user_followers_count', ascending=False).collect()

# Step 4: Find popular users

In [169]:
d5 =  d3.filter(d3.user_followers_count > 5000)
d6 = d5.orderBy('user_followers_count', ascending=False)
print((d6.count(), len(d6.columns)))

(3304, 3)


In [170]:
d6.show()

+-----------+--------------------+-----+
|user_id_str|user_followers_count|text_|
+-----------+--------------------+-----+
|        667|          1969294734|     |
|        259|          1921566948|     |
|        192|          1904765227|     |
|         75|          1883058216|     |
|        104|          1878410551|     |
|        348|          1873286317|     |
|        308|          1864643978|     |
|        338|          1830107652|     |
|        882|          1666728474|     |
|       1811|          1656420602|     |
|         38|          1653560462|     |
|         12|          1653412544|     |
|        149|          1642580719|     |
|       2081|          1620541724|     |
|        253|          1618351628|     |
|        153|          1608877050|     |
|        358|          1540800860|     |
|         59|          1539664699|     |
|        417|          1531752193|     |
|        237|          1518827550|     |
+-----------+--------------------+-----+
only showing top

# Step 5: Find top words and save the results

In [0]:
text_li = d6.select('text_')
text1 = text_li.toPandas()

In [0]:
#converting dataframe to string
text2 = text1.to_string()
print(text2)

In [0]:
# Delete all the numbers in the file
text3 = ''.join([i for i in text2 if not i.isdigit()])
text3

In [0]:
#removing punctuaton marks
text4 = text3.translate(str.maketrans('','',string.punctuation))
text4

In [0]:
#Converting file to lower case
text5 = text4.lower()
text5

In [0]:
#splitting
text6=text5.split()
text6

In [140]:
#Find all the URL's
urls=[]
for i in text6:
  if(i.startswith('http')==True or i.startswith('www')==True):
    urls.append(i)
print(urls)

['httpstco', 'httpstcomkutwflj', 'httpstcoiqgmcwxjjq', 'httpstcoouvfwqjc', 'https', 'httpstcopp', 'httpstc', 'https', 'httpst', 'httpstco', 'httpstcofrpzm', 'httpst', 'https', 'https', 'httpstcojvyw', 'httpstcog', 'httpstcotmptn', 'httpstcolnyavbgse', 'httpstcorowliruv', 'https', 'httpstcoruktweip', 'httpst', 'httpstcouolqawz', 'httpstcosopbmjrzf', 'https', 'httpstcoehsomrsde', 'httpstcoocazcj', 'httpstco', 'httpstcomkjouknak']


In [0]:
#Removing all URL's
for i in urls:
  text6.remove(i)

In [142]:
print(len(text6))

15453


In [0]:
#Adding value 1 to each element
dataP = []
for item in text6:
  temp = item #+' ' + str(1)
  dataP.append(temp)

dataP

In [0]:
#sorting the data pairs
dataP.sort()
#print(dataP)
#dataP

In [0]:
#Counting the words
counts = Counter(dataP)
counts

In [0]:
most_common_words = counts.most_common()
most_common_words

In [0]:
df_final = pd.DataFrame(most_common_words) 
pd.set_option('display.max_rows', 2576)

In [168]:
df_final

,0,1
0,amazonhelp,694
1,amazon,673
2,i,575
3,my,391
4,a,382
5,to,358
6,the,280
7,you,239
8,is,222
9,for,218
